In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Chamando funções (*functions calling*) com a Vertex AI Gemini API

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Executar no Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Ver no GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Abrir no Workbench da Vertex AI
    </a>
  </td>
</table>


## Visão geral

### Gêmeos

Gemini é uma família de modelos generativos de IA desenvolvidos pelo Google DeepMind e projetados para casos de uso multimodais.

### Chamando funções do Gemini

[Chamada de função](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) permite que os desenvolvedores criem uma descrição de uma função no código e depois passem essa descrição para uma linguagem modelo em uma solicitação. A resposta do modelo inclui o nome de uma função que corresponde à descrição e os argumentos para chamá-la.

A chamada de função é semelhante às [Extensões da Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/extensions/overview), pois ambas geram informações sobre funções. A diferença entre eles é que a chamada de função retorna dados JSON com o nome de uma função e os argumentos a serem usados em seu código, enquanto as extensões Vertex AI retornam a função e a chamam para você.

### Objetivos

Neste tutorial, você aprenderá como usar a API Vertex AI Gemini com o SDK Vertex AI para Python para fazer chamadas de função por meio do modelo Gemini Pro (`gemini-pro`).

Você executará as seguintes tarefas:

- Instalar o SDK da Vertex AI para Python
- Use a API Vertex AI Gemini para interagir com o modelo Gemini Pro (`gemini-pro`):
     - Gere chamadas de função a partir de um prompt de texto para obter a previsão do tempo para um determinado local
     - Gere chamadas de função a partir de um prompt de texto e chame uma API externa para geocodificar endereços
     - Gere chamadas de função a partir de um prompt de chat para ajudar usuários de varejo

### Custos

Este tutorial usa os seguintes componentes de Google Cloud que podem gerar custos em sua fatura:

- Vertex AI

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e use a [calculadora de preços](https://cloud.google.com/products/calculator/) para gerar uma estimativa de custo com base no uso projetado.

## Primeiros passos

### Instale a SDK da Vertex AI
**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# !pip3 install --upgrade --user google-cloud-aiplatform

### **Reinicie o kernel do seu jupyter notebook** 

Como a instalação está sendo realizada com a opção `--user` é necessário reiniciar o kernel para que os novos módulos se tornem acessíveis.

**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel do notebook está sendo reiniciado. Por favor aguarde este processo ser finalizado antes de continuar com os próximos passos. ⚠️</b>
</div>

### **Somente para uso no Colab - Autentique o seu ambiente de notebook** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para realizar a autenticação da sua sessão de notebook com a Google Cloud Esse passo é importante **para utilização no Colab** para garantir que as chamadas a APIs de Google Cloud funcionem sem problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Somente para uso no Colab - defina o projeto Google Cloud a ser utilizado** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para definir qual projeto Google Cloud será utilizado pelo Colab na execução deste notebook. Senão, siga para as próximas instruções.

In [ ]:
# if "google.colab" in sys.modules:
#     # Define project information
#     PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#     LOCATION = "us-central1"  # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importando as bibliotecas necessárias

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

## Por que chamar funções?

Ao trabalhar com um modelo de texto generativo, pode ser difícil forçar o LLM a fornecer respostas consistentes em um formato estruturado como JSON. A chamada de função facilita o trabalho com LLMs por meio de prompts e entradas não estruturadas, e faz com que o LLM retorne uma resposta estruturada que pode ser usada para chamar uma função externa.

Você pode pensar na chamada de função como uma forma de obter saída estruturada de prompts do usuário e definições de função, usar essa saída estruturada para fazer uma solicitação de API a um sistema externo e, em seguida, retornar a resposta da função ao LLM para gerar uma resposta ao usuário. Em outras palavras, a chamada de função no Gemini extrai parâmetros estruturados de textos não estruturados ou mensagens de usuários.

## Importe o modelo `Gemini 1.0 Pro`

O Gemini Pro (`gemini-1.0-pro`) ajuda na realização de tarefas utilizando linguagem natural, chats multiturno de texto e código e para a geração de código.

In [2]:
model = GenerativeModel("gemini-1.0-pro")

### Um exemplo simples de chamadas de função

Para começar, você usará a chamada de função para configurar uma solicitação de API meteorológica para que os usuários obtenham as condições atuais em um determinado local. Os parâmetros de função são especificados como um dicionário Python de acordo com o [formato de esquema JSON OpenAPI](https://spec.openapis.org/oas/v3.0.3#schemawr).

Considere um exemplo de função da API meteorológica que recebe um argumento para a localização do usuário, como em:

```python
def get_current_weather(location):
     ...
```

Você começará especificando uma declaração de função e os parâmetros necessários para fazer uma solicitação em nosso exemplo de API meteorológica:

In [3]:
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Busca informações de clima de um determinado local",
    parameters={
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "Location"
        }
    }
},
)

Você pode então definir uma ferramenta para o LLM chamar que inclua `get_current_weather_func`:

In [4]:
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

Você pode então instruir o modelo para gerar conteúdo, incluir a `tool` que acabou de criar, para gerar uma resposta:

In [5]:
prompt = "Como está o tempo em Belém?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[weather_tool],
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "Bel\303\251m"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 23
  candidates_token_count: 8
  total_token_count: 31
}

You can inspect the function call portion of the response:

In [6]:
response.candidates[0].content.parts[0].function_call

name: "get_current_weather"
args {
  fields {
    key: "location"
    value {
      string_value: "Bel\303\251m"
    }
  }
}

A resposta inclui uma assinatura de função que pode ser usada para chamar a API meteorológica. Neste ponto, você tem tudo o que precisa para formar um corpo de solicitação e fazer uma chamada de API para um sistema externo. Bom trabalho!

### Um exemplo mais complexo de chamadas de funções

Neste exemplo, você gerará uma chamada de função que possui uma estrutura mais complexa. Você usará a resposta da função para fazer uma chamada de API que converte um endereço em coordenadas de latitude e longitude.

Comece definindo uma declaração de função dentro de uma ferramenta:

In [7]:
get_location = FunctionDeclaration(
    name="get_location",
    description="Busque a latitude e a longitude para uma dada localização",
    parameters={
    "type": "object",
    "properties": {
        "poi": {
            "type": "string",
            "description": "Point of interest"
        },
        "street": {
            "type": "string",
            "description": "Street name"
        },
        "city": {
            "type": "string",
            "description": "City name"
        },
        "county": {
            "type": "string",
            "description": "County name"
        },
        "state": {
            "type": "string",
            "description": "State name"
        },
        "country": {
            "type": "string",
            "description": "Country name"
        },
        "postal_code": {
            "type": "string",
            "description": "Postal code"
        },
    },
},
)

location_tool = Tool(
    function_declarations=[get_location],
)

Agora você pode chamar o modelo para gerar uma resposta:

In [8]:
prompt = """
Quero obter as coordenadas lat/lon do seguinte endereço:
1600 Amphitheatre Pkwy, Mountain View, CA 94043, US
"""

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[location_tool],
)
response.candidates[0].content.parts[0]

function_call {
  name: "get_location"
  args {
    fields {
      key: "city"
      value {
        string_value: "Mountain View"
      }
    }
    fields {
      key: "country"
      value {
        string_value: "US"
      }
    }
    fields {
      key: "postal_code"
      value {
        string_value: "94043"
      }
    }
    fields {
      key: "state"
      value {
        string_value: "CA"
      }
    }
    fields {
      key: "street"
      value {
        string_value: "1600 Amphitheatre Pkwy"
      }
    }
  }
}

Agora você pode extrair os resultados da resposta da função e fazer uma chamada de API:

In [9]:
x = response.candidates[0].content.parts[0].function_call.args

url = "https://nominatim.openstreetmap.org/search?"
for i in x:
    url += '{}="{}"&'.format(i, x[i])
url += "format=json"

x = requests.get(url)
content = x.json()
content

[{'place_id': 377680635,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 2192620021,
  'lat': '37.4217636',
  'lon': '-122.084614',
  'class': 'office',
  'type': 'it',
  'place_rank': 30,
  'importance': 0.6949356759210291,
  'addresstype': 'office',
  'name': 'Google Headquarters',
  'display_name': 'Google Headquarters, 1600, Amphitheatre Parkway, Mountain View, Santa Clara County, California, 94043, United States',
  'boundingbox': ['37.4217136', '37.4218136', '-122.0846640', '-122.0845640']}]

Você foi capaz de construir uma função e uma ferramenta que o LLM usou para gerar os parâmetros necessários para uma chamada de função e, em seguida, fez a chamada de função para obter as coordenadas do local especificado.

Aqui usamos a [API OpenStreetMap Nominatim](https://nominatim.openstreetmap.org/ui/search.html) para geocodificar um endereço para facilitar o uso e o aprendizado neste notebook. Se você estiver trabalhando com grandes quantidades de mapas ou dados de geolocalização, poderá usar a [API de geocodificação do Google Maps](https://developers.google.com/maps/documentation/geocoding).

### Chamadas de função em interações de chat

Neste exemplo, você usará o modelo de chat no Gemini para ajudar os clientes a obter informações sobre os produtos de uma loja.

Você começará definindo várias funções em uma ferramenta para obter informações sobre produtos, obter a localização das lojas e fazer um pedido:

In [10]:
get_product_info_func = FunctionDeclaration(
    name="get_product_sku",
    description="Busque a SKU de um produto",
    parameters={
    "type": "object",
    "properties": {
        "product_name": {
            "type": "string",
            "description": "Nome do produto"
        }
    }
},
)

get_store_location_func = FunctionDeclaration(
    name="get_store_location",
    description="Busque a localização da loja mais próxima",
    parameters={
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "Localização"
        }
    }
},
)

place_order_func = FunctionDeclaration(
    name="place_order",
    description="Faça um pedido de compra",
    parameters={
    "type": "object",
    "properties": {
        "product": {
            "type": "string",
            "description": "Nome do produto"
        },
        "account": {
            "type": "integer",
            "description": "Informações da conta"
        },
        "address": {
            "type": "string",
            "description": "Endereço de entrega"
        }
    }
},
)

retail_tool = Tool(
    function_declarations=[get_product_info_func, 
                           get_store_location_func, 
                           place_order_func,
                          ],
)

Observe que você também pode usar chamadas de função em uma sessão de chat multiturno e pode especificar ferramentas ao criar um modelo para evitar ter que enviá-las em cada solicitação:

In [11]:
model = GenerativeModel("gemini-1.0-pro", 
                        generation_config={"temperature": 0},
                        tools=[retail_tool])
chat = model.start_chat()

In [12]:
prompt = """
Você tem o Pixel 8 Pro em estoque?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_product_sku"
  args {
    fields {
      key: "product_name"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

Como esperado, a resposta inclui uma chamada de função estruturada que podemos usar para nos comunicarmos com sistemas externos.

Na realidade, você executaria chamadas de função em um sistema ou banco de dados externo. Como este notebook se concentra na capacidade de extrair parâmetros de função e gerar chamadas de função, você usará dados simulados para alimentar as respostas ao modelo, em vez de usar um servidor de API real.

In [13]:
# É aqui que você faria uma solicitação de API para retornar o status do pedido.
# Use dados sintéticos para simular uma resposta de uma API externa.

api_response = {"sku": "GA04834-US", "em_stock": "sim"}

Agora, vamos incluir detalhes da chamada de API externa e gerar uma resposta ao usuário:

In [14]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_sku",
        response={
            "content": api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "Sim, temos o Pixel 8 Pro em estoque."

Em seguida, o usuário pode perguntar onde pode comprar um telefone em uma loja próxima:

In [15]:
prompt = """
Onde posso comprá-lo perto de Mountain View, CA?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_store_location"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}

Obtemos uma resposta com outra chamada de função estruturada, desta vez configurada para usar uma função diferente da nossa ferramenta.

In [16]:
# É aqui que você faria uma solicitação de API para obter a localização da loja mais próxima do usuário.
# Use dados sintéticos para simular uma resposta de uma API externa.

api_response = {"store": "1600 Amphitheatre Pkwy, Mountain View, CA 94043, US"}

Again, let's include details from the external API call and generate a response to the user:

In [17]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content":  api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "A loja mais pr\303\263xima fica em 1600 Amphitheatre Pkwy, Mountain View, CA 94043, US."

Por fim, o usuário pode solicitar o pedido de um telefone e enviá-lo para um endereço:

In [18]:
prompt = """
Gostaria de encomendar um Pixel 8 Pro e enviá-lo para 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "place_order"
  args {
    fields {
      key: "address"
      value {
        string_value: "1155 Borregas Ave, Sunnyvale, CA 94089"
      }
    }
    fields {
      key: "product"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

Extraímos o produto desejado e o endereço do usuário, buscamos o número da conta e agora podemos chamar uma API para fazer o pedido:

In [19]:
# É aqui que você faria uma solicitação de API para retornar o status do pedido.
# Use dados sintéticos para simular uma carga útil de resposta de uma API externa.

api_response = {"status_pagamento": "pago", "numero_pedido": 12345, "prazo_entrega": "2 dias"}

Como antes, vamos incluir detalhes da chamada de API externa e gerar uma resposta ao usuário:

In [20]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content":  api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "Ok, fiz o seu pedido. O n\303\272mero do pedido \303\251 12345. O prazo de entrega \303\251 de 2 dias e o pagamento j\303\241 foi processado."

Você teve uma conversa de várias voltas com o Gemini, gerou chamadas de função, tratou da passagem (simulada) de dados de volta para o modelo e gerou mensagens que fizeram uso das respostas da função.